In [ ]:
from google.colab import files    #its local access
uploaded = files.upload()

In [ ]:
from google.colab import drive       # Drive Mount
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#   /content/drive/MyDrive/Project5:img/Dataset/images/data          # place of data

In [6]:
# ....................................
# Install and Import Libraries
# ....................................

import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    MobileNetV2, ResNet50, VGG16, EfficientNetB0
)

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# ....................................
# Data Preprocessing
# ....................................

IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_dir = "/content/drive/MyDrive/Project5:img/Dataset/images/data/train"
test_dir = "/content/drive/MyDrive/Project5:img/Dataset/images/data/test"

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

num_classes = train_data.num_classes

Found 6225 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.


Found 6225 images belonging to 11 classes.

Found 3187 images belonging to 11 classes.

In [ ]:
# ....................................
# MODEL 1: CUSTOM CNN
# ....................................

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.fit(train_data, validation_data=test_data, epochs=1)
cnn_acc = cnn_model.evaluate(test_data)[1]

390/390 ━━━━━━━━━━━━━━━━━━━━ 874s 2s/step - accuracy: 0.3461 - loss: 2.5515 - val_accuracy: 0.6144 - val_loss: 1.0730
200/200 ━━━━━━━━━━━━━━━━━━━━ 90s 451ms/step - accuracy: 0.6150 - loss: 1.0803


Model 2 : accuracy--0.6150

In [ ]:
# ....................................
# MODEL 2: MOBILENETV2
# ....................................

base_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_mobilenet.trainable = False

x = GlobalAveragePooling2D()(base_mobilenet.output)
output = Dense(num_classes, activation='softmax')(x)

mobilenet_model = Model(base_mobilenet.input, output)

mobilenet_model.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

mobilenet_model.fit(train_data, validation_data=test_data, epochs=1)
mobilenet_acc = mobilenet_model.evaluate(test_data)[1]

390/390 ━━━━━━━━━━━━━━━━━━━━ 501s 1s/step - accuracy: 0.7426 - loss: 0.9312 - val_accuracy: 0.9771 - val_loss: 0.1246
200/200 ━━━━━━━━━━━━━━━━━━━━ 144s 722ms/step - accuracy: 0.9764 - loss: 0.1268


Model 2 : accuracy -- 0.9764

In [ ]:
# ....................................
# MODEL 3: RESNET50
# ....................................

base_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_resnet.trainable = False

x = GlobalAveragePooling2D()(base_resnet.output)
output = Dense(num_classes, activation='softmax')(x)

resnet_model = Model(base_resnet.input, output)

resnet_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

resnet_model.fit(train_data, validation_data=test_data, epochs=1)
resnet_acc = resnet_model.evaluate(test_data)[1]

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
390/390 ━━━━━━━━━━━━━━━━━━━━ 1824s 5s/step - accuracy: 0.1673 - loss: 2.3170 - val_accuracy: 0.2052 - val_loss: 2.1502
200/200 ━━━━━━━━━━━━━━━━━━━━ 591s 3s/step - accuracy: 0.1963 - loss: 2.1665


MODEL 3: RESNET50 Accuracy : 0.1963

In [ ]:
# ....................................
# MODEL 4: VGG16
# ....................................

base_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_vgg.trainable = False

x = GlobalAveragePooling2D()(base_vgg.output)
output = Dense(num_classes, activation='softmax')(x)

vgg_model = Model(base_vgg.input, output)

vgg_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

vgg_model.fit(train_data, validation_data=test_data, epochs=1)
vgg_acc = vgg_model.evaluate(test_data)[1]

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


390/390 ━━━━━━━━━━━━━━━━━━━━ 6460s 17s/step - accuracy: 0.3281 - loss: 2.0291 - val_accuracy: 0.7022 - val_loss: 1.4400
200/200 ━━━━━━━━━━━━━━━━━━━━ 1671s 8s/step - accuracy: 0.7073 - loss: 1.4270


MODEL 4: VGG16 Accuracy : 0.7073

In [5]:
# ....................................
# MODEL 5: EFFICIENTNETB0
# ....................................

base_eff = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_eff.trainable = False

x = GlobalAveragePooling2D()(base_eff.output)
output = Dense(num_classes, activation='softmax')(x)

eff_model = Model(base_eff.input, output)

eff_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

eff_model.fit(train_data, validation_data=test_data, epochs=1)
eff_acc = eff_model.evaluate(test_data)[1]

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


390/390 ━━━━━━━━━━━━━━━━━━━━ 2412s 6s/step - accuracy: 0.1496 - loss: 2.3504 - val_accuracy: 0.0957 - val_loss: 2.3507
200/200 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.0986 - loss: 2.3484


MODEL 5: EFFICIENTNETB0 Accuracy : 0.0986

In [7]:
# ....................................
# COMPARE ALL 5 MODELS
# ....................................

# results = pd.DataFrame({
#     "Model": ["CNN", "MobileNetV2", "ResNet50", "VGG16", "EfficientNetB0"],
#     "Accuracy": [cnn_acc, mobilenet_acc, resnet_acc, vgg_acc, eff_acc]
# })

# All Model Accuracy entered in the manual

results = {
    "Model": ["CNN", "MobileNetV2", "ResNet50", "VGG16", "EfficientNetB0"],
    "Accuracy": [0.6150, 0.9764, 0.1963, 0.7073, 0.0986]
}

results


{'Model': ['CNN', 'MobileNetV2', 'ResNet50', 'VGG16', 'EfficientNetB0'],
 'Accuracy': [0.615, 0.9764, 0.1963, 0.7073, 0.0986]}

In [ ]:
# ....................................
# SELECT BEST MODEL AUTOMATICALLY
# ....................................

best_model_name = results.loc[results["Accuracy"].idxmax(), "Model"]
print("Best Model:", best_model_name)

if best_model_name == "CNN":
    best_model = cnn_model
elif best_model_name == "MobileNetV2":
    best_model = mobilenet_model
elif best_model_name == "ResNet50":
    best_model = resnet_model
elif best_model_name == "VGG16":
    best_model = vgg_model
else:
    best_model = eff_model

In [8]:
# ....................................
# Build Model (Transfer Learning)
# ....................................

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,371 (9.24 MB)

 Trainable params: 165,387 (646.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [9]:
# ....................................
# Train Model
# ....................................

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=1
)

390/390 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.8322 - loss: 0.5899 - val_accuracy: 0.9881 - val_loss: 0.0574


In [12]:
# ....................................
# Save Model
# ....................................

model.save("fish_classifier.h5")
print("Model saved successfully")

Model saved successfully


In [4]:
# ....................................
# Test Single Image Prediction              #read full and then run
# ....................................

from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/drive/MyDrive/Project5:img/Dataset/images/data/test/fish sea_food black_sea_sprat/00JL3HEP2KW2.jpg"  # change image path

img = image.load_img(img_path, target_size=IMG_SIZE)
img = image.img_to_array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)

pred = model.predict(img)
class_names = list(train_data.class_indices.keys())

print("Predicted Class:", class_names[np.argmax(pred)])

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
!pip install streamlit

^C


In [3]:
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image

# -----------------------------
# PAGE CONFIG
# -----------------------------
st.set_page_config(
    page_title="Fish Image Classifier",
    page_icon="🐟",
    layout="centered"
)

# -----------------------------
# CUSTOM CSS
# -----------------------------
st.markdown("""
<style>
body {
    background-color: #0e1117;
}
.main {
    background-color: #0e1117;
    color: white;
}
.stButton>button {
    background-color: #1f77b4;
    color: white;
    border-radius: 10px;
    padding: 10px 20px;
}
.stFileUploader {
    border: 2px dashed #1f77b4;
    border-radius: 10px;
}
.result-box {
    background-color: #1e1e1e;
    padding: 20px;
    border-radius: 12px;
    margin-top: 20px;
    text-align: center;
    font-size: 20px;
}
</style>
""", unsafe_allow_html=True)

# -----------------------------
# TITLE
# -----------------------------
st.markdown("<h1 style='text-align:center;'>🐠 Fish Image Classification App</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align:center;'>Upload an image and get instant prediction</p>", unsafe_allow_html=True)

# -----------------------------
# LOAD MODEL
# -----------------------------
@st.cache_resource
def load_trained_model():
    return load_model("model.h5")   # change model path if needed

model = load_trained_model()

# -----------------------------
# CLASS NAMES
# -----------------------------
class_names = [
    "black_sea_sprat",
    "gilt_head_bream",
    "horse_mackerel",
    "red_mullet",
    "sea_bass",
    "shrimp",
    "striped_red_mullet",
    "trout"
]

IMG_SIZE = (224, 224)

# -----------------------------
# FILE UPLOADER
# -----------------------------
uploaded_file = st.file_uploader("📤 Upload a Fish Image", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    img = Image.open(uploaded_file).convert("RGB")
    st.image(img, caption="📸 Uploaded Image", use_container_width=True)

    # -----------------------------
    # PREPROCESS IMAGE
    # -----------------------------
    img = img.resize(IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # -----------------------------
    # PREDICT BUTTON
    # -----------------------------
    if st.button("🔍 Predict"):
        with st.spinner("Analyzing image..."):
            pred = model.predict(img_array)
            predicted_class = class_names[np.argmax(pred)]
            confidence = np.max(pred) * 100

        # -----------------------------
        # RESULT
        # -----------------------------
        st.markdown(
            f"""
            <div class="result-box">
                🐟 <b>Predicted Class:</b> {predicted_class}<br>
                📊 <b>Confidence:</b> {confidence:.2f}%
            </div>
            """,
            unsafe_allow_html=True
        )

# -----------------------------
# FOOTER
# -----------------------------
st.markdown("<hr>", unsafe_allow_html=True)
st.markdown(
    "<p style='text-align:center;'>Built with ❤️ using Streamlit & TensorFlow</p>",
    unsafe_allow_html=True
)

ModuleNotFoundError: No module named 'tensorflow'